In [1]:
from epi_model.common_imports import *

In [2]:
from simulation_manager import simulation_manager

In [3]:
from frozendict import frozendict as fzd

In [4]:
names = [
    'HS1',
    'HS2',
    'synthetic_3000'
]

# models

In [5]:
modelsets = {}

In [6]:
for name in names:
    print(f'loading {name}')
    sim_m = simulation_manager()
    sim_m.load_models(name)
    
    modelsets[name] = sim_m

loading HS1
loading HS2
loading synthetic_3000


# various params

In [7]:
comparisons = {
    'VACCINATE_P': [0.05, 0.10, 0.20, 0.50],
    'R0_mean': [1, 2.5, 4],
    'INITIAL_INFECT': [5, 10, 20]
}

In [8]:
defaults = {
    'R0_mean': 2.5,
    'VACCINATE_P': 0.20,
    'INITIAL_INFECT': 20
}

In [9]:
titles = {
    "HS1": "HS-1",
    "HS2": "HS-2",
    "synthetic_3000": "the synthetic network",
}
titlestr = sorted(titles)

In [10]:
strat_print = {
    'friend': "NR",
    'friendHighDegRandTop5': "NP(N=5)",
    'friendHighDegRandTop3': "NP(N=3)",
    'friendHighDegNormalErr20': "NP(ε=20)",
    'friendHighDegNormalErr10': "NP(ε=10)",
    'friendHighDeg': "NP",
    'friendHighDegChain': "NP(c)",
    'targeted': "D"
}

# some first descriptives

In [61]:
def comma_and(X):
    return ", ".join( X[:-1] ) + ", and " + X[-1]

In [62]:
desc = """
First we present the relative effectiveness of these vaccination strategies under one scenario. 
That is, with the 
    average infectiousness R<sub>0</sub> = 2.5 (with coefficient of variance = 0.2), and 
    initially infecting 20 individuals."""
#,     and vaccinating 20% of the population.

d = dict(defaults)
d['strat'] = 'none'
d = fzd(d)

means = []
for ts in titlestr:
    m = modelsets[ts]
    inf = m.info(d)
    means.append(f"{inf['mean']:0,.1f} +/- {inf['stderr']:0.1f} of the {inf['init_sus']} remaining susceptible in {titles[ts]} ({inf['mean']/inf['init_sus']:0.1%})")
means = comma_and( means )

desc += f"""
The average numbers of individuals infected in each context when we do not vaccinate at all, across 300 simulations, are {means}."""

In [63]:
pcts = []
for ts in titlestr:
    m = modelsets[ts]
    inf = m.info(d)
    pct = np.sum( np.array(inf['ninf']) >= inf['init_sus']*0.25 ) / inf['Nsims']
    pcts.append( f'{pct:0.1%} ({titles[ts]})' )
pcts = comma_and( pcts )

desc += f"""
More than 25% of the population is infected in {pcts} of simulations."""

In [64]:
display(Markdown(desc))


First we present the relative effectiveness of these vaccination strategies under one scenario. 
That is, with the 
    average infectiousness R<sub>0</sub> = 2.5 (with coefficient of variance = 0.2), and 
    initially infecting 20 individuals.
The average numbers of individuals infected in each context when we do not vaccinate at all, across 300 simulations, are 28.0 +/- 0.6 of the 290 remaining susceptible in HS-1 (9.7%), 149.8 +/- 3.3 of the 764 remaining susceptible in HS-2 (19.6%), and 267.6 +/- 10.6 of the 2980 remaining susceptible in the synthetic network (9.0%).
More than 25% of the population is infected in 0.2% (HS-1), 29.4% (HS-2), and 2.0% (the synthetic network) of simulations.

In [65]:
def err( inf ):
    ci1, mn, ci2 = inf['P_sus_inf_rel_none_BS']
    
    # as long as they're symmetric within rounding
    #if (ci2-mn) - (mn-ci1) > 0.005:
    #    print("warning: asymmetry above half a percent", ci1+ci2-2*mn, ci1, mn, ci2)
    print("asymmetry = ", f"{ci1+ci2-2*mn:0.1%}")
    e = (ci2 - ci1) / 2
    return f"{e:0.1%}"

In [66]:
d2 = dict(defaults)
d2['strat'] = 'rand'
d2 = fzd(d2)

pcts = []
for ts in titlestr:
    m = modelsets[ts]
    
    inf1 = m.info(d)
    inf2 = m.info(d2)
    
    pct = 1 - inf2['P_sus_inf'] / inf1['P_sus_inf']
    
    #pcts.append( f'{pct:0.1%} ({titles[ts]}; 95% CI {1-ci1:0.1%} - {1-ci2:0.1%})' )
    #pcts.append( f'{pct:0.1%} +/- {err(inf2)} ({titles[ts]})' )
    
    ci2, mn, ci1 = inf2['P_sus_inf_rel_none_BS']
    pcts.append( f'{pct:0.1%} ({titles[ts]}; {1-ci1:0.1%} - {1-ci2:0.1%})' )
    
pcts = comma_and( pcts )

desc += f"""
Vaccinating 20% of the population randomly will decrease the average number of infections after 100 days by {pcts}."""

In [67]:
display(Markdown(desc))


First we present the relative effectiveness of these vaccination strategies under one scenario. 
That is, with the 
    average infectiousness R<sub>0</sub> = 2.5 (with coefficient of variance = 0.2), and 
    initially infecting 20 individuals.
The average numbers of individuals infected in each context when we do not vaccinate at all, across 300 simulations, are 28.0 +/- 0.6 of the 290 remaining susceptible in HS-1 (9.7%), 149.8 +/- 3.3 of the 764 remaining susceptible in HS-2 (19.6%), and 267.6 +/- 10.6 of the 2980 remaining susceptible in the synthetic network (9.0%).
More than 25% of the population is infected in 0.2% (HS-1), 29.4% (HS-2), and 2.0% (the synthetic network) of simulations.
Vaccinating 20% of the population randomly will decrease the average number of infections after 100 days by 22.8% (HS-1; 18.3% - 26.9%), 52.9% (HS-2; 49.5% - 55.7%), and 62.3% (the synthetic network; 57.4% - 66.4%).

In [68]:
d3 = dict(defaults)
d3['strat'] = 'friendHighDeg'
d3 = fzd(d3)

pcts = []
for ts in titlestr:
    m = modelsets[ts]
    
    inf = m.info(d3)

    pct = 1 - inf['P_sus_inf_rel_rand']
    #var = inf['P_sus_inf_rel_rand_err']
    #pcts.append( f'{pct:0.1%} +/- {var:0.1%} ({titles[ts]})' )
    
    ci2, mn, ci1 = inf['P_sus_inf_rel_rand_BS']
    pcts.append( f'{pct:0.1%} ({titles[ts]}; {1-ci1:0.1%} - {1-ci2:0.1%})' )
    
pcts = comma_and( pcts )

desc += f"""
And crucially for the purposes of this paper, nomination of most popular contacts (NP) does significantly better than random vaccination. 
Vaccinating 20% of the population in this way decreases the average number of infections after 100 days by a *further* {pcts}, relative to random vaccination."""

In [69]:
display(Markdown(desc))


First we present the relative effectiveness of these vaccination strategies under one scenario. 
That is, with the 
    average infectiousness R<sub>0</sub> = 2.5 (with coefficient of variance = 0.2), and 
    initially infecting 20 individuals.
The average numbers of individuals infected in each context when we do not vaccinate at all, across 300 simulations, are 28.0 +/- 0.6 of the 290 remaining susceptible in HS-1 (9.7%), 149.8 +/- 3.3 of the 764 remaining susceptible in HS-2 (19.6%), and 267.6 +/- 10.6 of the 2980 remaining susceptible in the synthetic network (9.0%).
More than 25% of the population is infected in 0.2% (HS-1), 29.4% (HS-2), and 2.0% (the synthetic network) of simulations.
Vaccinating 20% of the population randomly will decrease the average number of infections after 100 days by 22.8% (HS-1; 18.3% - 26.9%), 52.9% (HS-2; 49.5% - 55.7%), and 62.3% (the synthetic network; 57.4% - 66.4%).
And crucially for the purposes of this paper, nomination of most popular contacts (NP) does significantly better than random vaccination. 
Vaccinating 20% of the population in this way decreases the average number of infections after 100 days by a *further* 17.1% (HS-1; 12.1% - 22.0%), 17.3% (HS-2; 11.2% - 22.7%), and 59.5% (the synthetic network; 54.3% - 63.6%), relative to random vaccination.